Copyright 2024 - Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Gemini Pro - Using LangChain to integrate with Vertex AI Search for RAG architecture.

* This notebook explains how to use grounding service in Gemini Pro.
* Refer to https://cloud.google.com/vertex-ai/generative-ai/docs/grounding/overview
* Using Vertex AI Search :
  * https://cloud.google.com/vertex-ai/generative-ai/docs/grounding/overview#ground-private
* Refer to the Langchain website.
  * https://python.langchain.com/docs/integrations/retrievers/google_vertex_ai_search

# Configuration
## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest


In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 30.9 MB/s eta 0:00:00


In [2]:
%pip install --upgrade --quiet google-cloud-discoveryengine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.3 MB/s eta 0:00:00


In [15]:
%pip install --upgrade --quiet langchain langchain-google-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 20.9 MB/s eta 0:00:00


In [4]:
from IPython.display import display, Markdown

## Authentication to access to the GCP & Google drive

* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [5]:
#  For only colab to authenticate to get an access to the GCP.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

* Mount to the google drive to access the .ipynb files in the repository.

In [6]:
# To access contents in Google drive

if "google.colab" in sys.modules:
  from google.colab import drive
  drive.mount('/content/drive')

Mounted at /content/drive


# Execute the example
## Set the environment on GCP Project
* Configure project information
  * Model name : LLM model name : https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models
  * Project Id : prodect id in GCP
  * Region : region name in GCP

In [7]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [8]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part, Tool
import vertexai.generative_models as generative_models

# Grounding service is still in preview.
from vertexai.preview.generative_models import grounding

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)

# Access to the generative model.
model = GenerativeModel(MODEL_NAME)

### Configure Langchain function to integrate with the Vertex AI Search
* Currently using digital parser

In [27]:
from langchain.retrievers import GoogleVertexAIMultiTurnSearchRetriever
from langchain_google_community import VertexAISearchRetriever

DS_REGION="global"
DATA_STORE_ID = "it-law-digital-parser_1718024672666"

retriever = GoogleVertexAISearchRetriever(
    project_id=PROJECT_ID,
    location_id=DS_REGION,
    data_store_id=DATA_STORE_ID,
    max_documents=3,
    # max_extractive_answer_count=2,
    # get_extractive_answers=True,
    max_extractive_segment_count=1,
    get_extractive_segments=True,
    query_expansion_condition=2,
    engine_data_type = 0
)

### Query context from the Vertex AI Search

In [28]:
query = "침해사고의 원인 분석에 대해서 설명해주세요."

result = retriever.get_relevant_documents(query)

context ={}

for doc in result:
    print(doc.json)
    context[doc.metadata['source']] = doc.page_content
    print("\n\n======================")
    print(doc.metadata['source'])
    print(doc.page_content)

print(context)



<bound method BaseModel.json of Document(page_content='제48조의4(침해사고의 원인 분석 등) ① 정보통신서비스 제공자 등 정보통신망을 운영하는 자는 침\n해사고가 발생하면 침해사고의 원인을 분석하고 그 결과에 따라 피해의 확산 방지를 위하여 사고\n대응, 복구 및 재발 방지에 필요한 조치를 하여야 한다. <개정 2022. 6. 10.>\n② 과학기술정보통신부장관은 정보통신서비스 제공자의 정보통신망에 침해사고가 발생하면 그\n침해사고의 원인을 분석하고 피해 확산 방지, 사고대응, 복구 및 재발 방지를 위한 대책을 마련\n하여 해당 정보통신서비스 제공자(공공기관등은 제외한다)에게 필요한 조치를 이행하도록 명령\n할 수 있다.<신설 2022. 6. 10., 2024. 2. 13.>\n③ 과학기술정보통신부장관은 제2항에 따른 조치의 이행 여부를 점검하고, 보완이 필요한 사항\n에 대하여 해당 정보통신서비스 제공자에게 시정을 명할 수 있다.<신설 2024. 2. 13.>\n④ 과학기술정보통신부장관은 정보통신서비스 제공자의 정보통신망에 중대한 침해사고가 발생\n한 경우 제2항에 따른 원인 분석 및 대책 마련을 위하여 필요하면 정보보호에 전문성을 갖춘 민\nㆍ관합동조사단을 구성하여 그 침해사고의 원인 분석을 할 수 있다.<개정 2013. 3. 23., 2017. 7. 26.,\n2022. 6. 10., 2024. 2. 13.>\n⑤ 과학기술정보통신부장관은 제2항에 따른 침해사고의 원인 분석 및 대책 마련을 위하여 필요\n하면 정보통신서비스 제공자에게 정보통신망의 접속기록 등 관련 자료의 보전을 명할 수 있다.\n<개정 2013. 3. 23., 2017. 7. 26., 2022. 6. 10., 2024. 2. 13.>\n⑥ 과학기술정보통신부장관은 제2항에 따른 침해사고의 원인 분석 및 대책 마련을 하기 위하여\n필요하면 정보통신서비스 제공자에게 침해사고 관련 자료의 제출을 요구할 수 있으며, 중대한\n침해사고의 경우 소속 공무원 

### Create model object with LangChain API

In [29]:
from langchain_google_vertexai.llms import VertexAI

gemini_pro = VertexAI( model_name = MODEL_NAME,
                  project=PROJECT_ID,
                  location=REGION,
                  verbose=True,
                  streaming=False,
                  temperature = 0.2,
                  top_p = 1,
                  top_k = 40
                 )

### Request to the LLM

In [30]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate.from_template("""

  당신은 법률을 상담해주는 AI 어시스턴트입니다.
  아래 Question 에 대해서 반드시 Context에 있는 개별 내용을 기반으로 단계적으로 추론해서 근거를 설명하고 쉽게 답변해주세요.
  Context : {context}
  Question : {question}

  """)

prompt = prompt.format(context=context,
                       question=query)

print(f"Prompt : {prompt}")

response = gemini_pro.invoke(prompt)
display(Markdown(response))


Prompt : 

  당신은 법률을 상담해주는 AI 어시스턴트입니다.
  아래 Question 에 대해서 반드시 Context에 있는 개별 내용을 기반으로 단계적으로 추론해서 근거를 설명하고 쉽게 답변해주세요.
  Context : {'gs://it_laws_kr/law_pdf/정보통신망 이용촉진 및 정보보호 등에 관한 법률(법률)(제20069호)(20240123).pdf': '제48조의4(침해사고의 원인 분석 등) ① 정보통신서비스 제공자 등 정보통신망을 운영하는 자는 침\n해사고가 발생하면 침해사고의 원인을 분석하고 그 결과에 따라 피해의 확산 방지를 위하여 사고\n대응, 복구 및 재발 방지에 필요한 조치를 하여야 한다. <개정 2022. 6. 10.>\n② 과학기술정보통신부장관은 정보통신서비스 제공자의 정보통신망에 침해사고가 발생하면 그\n침해사고의 원인을 분석하고 피해 확산 방지, 사고대응, 복구 및 재발 방지를 위한 대책을 마련\n하여 해당 정보통신서비스 제공자(공공기관등은 제외한다)에게 필요한 조치를 이행하도록 명령\n할 수 있다.<신설 2022. 6. 10., 2024. 2. 13.>\n③ 과학기술정보통신부장관은 제2항에 따른 조치의 이행 여부를 점검하고, 보완이 필요한 사항\n에 대하여 해당 정보통신서비스 제공자에게 시정을 명할 수 있다.<신설 2024. 2. 13.>\n④ 과학기술정보통신부장관은 정보통신서비스 제공자의 정보통신망에 중대한 침해사고가 발생\n한 경우 제2항에 따른 원인 분석 및 대책 마련을 위하여 필요하면 정보보호에 전문성을 갖춘 민\nㆍ관합동조사단을 구성하여 그 침해사고의 원인 분석을 할 수 있다.<개정 2013. 3. 23., 2017. 7. 26.,\n2022. 6. 10., 2024. 2. 13.>\n⑤ 과학기술정보통신부장관은 제2항에 따른 침해사고의 원인 분석 및 대책 마련을 위하여 필요\n하면 정보통신서비스 제공자에게 정보통신망의 접속기록 등 관련

정보통신망법 제48조의4에 따르면 정보통신서비스 제공자 등 정보통신망을 운영하는 자는 침해사고가 발생하면 침해사고의 원인을 분석해야 합니다. 

**구체적으로 다음과 같은 내용을 담고 있습니다.**

1. **침해사고 발생 시 원인 분석 의무:** 정보통신서비스 제공자는 침해사고 발생 시 그 원인을 분석해야 합니다.
2. **과학기술정보통신부장관의 분석 권한:** 과학기술정보통신부장관은 정보통신서비스 제공자의 정보통신망에 침해사고가 발생하면 그 원인을 분석하고 피해 확산 방지, 사고대응, 복구 및 재발 방지를 위한 대책을 마련할 수 있습니다.
3. **민·관합동조사단 구성:** 중대한 침해사고 발생 시 과학기술정보통신부장관은 정보보호에 전문성을 갖춘 민·관합동조사단을 구성하여 침해사고의 원인을 분석할 수 있습니다.
4. **자료 보전 명령:** 과학기술정보통신부장관은 침해사고의 원인 분석 및 대책 마련을 위해 정보통신서비스 제공자에게 정보통신망의 접속기록 등 관련 자료의 보전을 명할 수 있습니다.
5. **자료 제출 요구:** 과학기술정보통신부장관은 침해사고의 원인 분석 및 대책 마련을 위해 정보통신서비스 제공자에게 침해사고 관련 자료의 제출을 요구할 수 있으며, 중대한 침해사고의 경우 관계인의 사업장에 출입하여 침해사고 원인을 조사하도록 할 수 있습니다.

**즉, 침해사고 발생 시 정보통신서비스 제공자는 원인 분석을 통해 피해 확산 방지, 사고 대응, 복구 및 재발 방지에 필요한 조치를 취해야 합니다.** 
